# Fault-equivalent rewrites

This notebook serves as a practical guide to the fault-equivalent rewrites available in **PyZX**. It shows which rewrites are currently implemented and how they can be applied. 

Fault-equivalent rewrites are introduced in arXiv:2506.17181. An equivalent formulation, known as *distance-preserving rewrites*, is presented in arXiv:2410.17240. 

Examples are provided to illustrate each rewrite. The identity removal, push pauli, and colour-change rules are general, non-specific rewrites that are always fault-equivalent. Consequently, they are not included in this notebook.

In [28]:
import pyzx as zx
from pyzx import VertexType, EdgeType, Graph
from pyzx.graph.multigraph import Multigraph
from pyzx.graph.base import BaseGraph
from pyzx.rewrite_rules.unfuse_FE_rules import *
from pyzx.rewrite_rules.fuse_1_FE_rule import *
from pyzx.rewrite_rules.fuse_FE_rules import *

We start with the FE unfuse-1 rule:

In [29]:
def generate_zx_graph(N: int, W: int | None = None) -> BaseGraph:
    """
    Generates a ZX-diagram graph for the given N and W parameters.
    N can be any integer.
    """
    g = zx.Graph()
    # The position is calculated to keep the graph centered.
    v = g.add_vertex(VertexType.Z, (N - 1) / 2, 0)

    # Add N boundary vertices and connect them to the central spider
    for i in range(N):
        nv = g.add_vertex(VertexType.BOUNDARY, i, 1)
        g.add_edge((v, nv))

    return g, v

n=8
g, v = generate_zx_graph(n)

zx.draw(g)

unfuse_1_FE(g,v)
zx.draw(g)

The FE fuse-1 rule is the inverse of the FE unfuse-1 rule illustrated above. Starting from the diagram produced by the unfuse-1 rewrite, the fuse-1 rule takes the leftmost vertex as input and reconstructs the original ZX diagram.

Next, we illustrate the FE unfuse-4 rule. Again, the Fe fuse-4 rule is the inverse of this operation, which takes as input the four vertices that need to be merged (see the example of the fuse-5 rule) 

In [24]:
g = Graph()
v = g.add_vertex(VertexType.Z, 0, 0)
for i in range(4):
    nv = g.add_vertex(VertexType.BOUNDARY, 2 * (i // 2) - 1, 2 * (i % 2) - 1)
    g.add_edge((v, nv))

zx.draw(g)

unfuse_4_FE(g, v)
zx.draw(g)

We illustrate the FE fuse-5 rule below. By increasing `n` beyond 5, the example can be extended to demonstrate the more general fuse-n rule, which can be applied using `safe_fuse_n_2FE(g, z_vertices)`.

Note that this general fuse-n rule is only **2-fault-equivalent**. Full fault equivalence holds only for the cases `n = 4` and `n = 5`.

In [23]:
import math

g = Multigraph()

n = 5
radius_inner = 1.5
radius_outer = 3.0

z_vertices = []
b_vertices = []

for i in range(n):
    angle = 2 * math.pi * i / n

    z = g.add_vertex(VertexType.Z, radius_inner * math.sin(angle), radius_inner * math.cos(angle))
    z_vertices.append(z)

    b = g.add_vertex(VertexType.BOUNDARY, radius_outer * math.sin(angle), radius_outer * math.cos(angle))
    b_vertices.append(b)


for i in range(n):
    g.add_edge((z_vertices[i], z_vertices[(i + 1) % n]))

for z, b in zip(z_vertices, b_vertices):
    g.add_edge((z, b))

zx.draw(g)

safe_fuse_5_FE(g, z_vertices)
zx.draw(g)

Finally, we illustrate the FE Unfuse-2n (can only be applied to `n=even`) and the recursive unfuse rewrite. The fuse versions of these rewrites are not (yet) implemented in **Pyzx**.

In [33]:
g, v = generate_zx_graph(8, None)
zx.draw(g)

unfuse_2n_FE(g, v)
zx.draw(g)

In [37]:
g, v = generate_zx_graph(7, None)
zx.draw(g)

recursive_unfuse_FE(g, v)
zx.draw(g)

Setting some `w` value will apply the `w`-fault-equivalent unfuse_2n_w rewrite as defined in Proposition 6.16 of arXiv:2506.17181. 

In [39]:
w=3
g, v = generate_zx_graph(8, w)
zx.draw(g)

unfuse_2n_FE(g, v, w)
zx.draw(g)